In [1]:
import anndata as ad
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import pandas as pd
from scipy import stats
import torch
from os import sys
sys.path.append('..')
from VAE.VAE_model import VAE
from torch.autograd import Variable
from tqdm import tqdm


/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/utils.py:429: 

In [2]:
def gaussian_kernel(X, Y, sigma=1.0):

    XX = np.sum(X ** 2, axis=1, keepdims=True)
    YY = np.sum(Y ** 2, axis=1, keepdims=True)
    XY = np.dot(X, Y.T)
    dist = XX + YY.T - 2 * XY
    return np.exp(-dist / (2 * sigma ** 2))


def compute_mmd(X, Y, sigma=1.0):

    n = X.shape[0]
    m = Y.shape[0]

    K_XX = gaussian_kernel(X, X, sigma)
    K_YY = gaussian_kernel(Y, Y, sigma)
    K_XY = gaussian_kernel(X, Y, sigma)

    mmd_squared = (np.sum(K_XX) / (n * n) + np.sum(K_YY) / (m * m) - 2 * np.sum(K_XY) / (n * m))
    return np.sqrt(mmd_squared)

def load_VAE():
    autoencoder = VAE(
        num_genes=17789,
        device='cuda',
        seed=0,
        loss_ae='mse',
        hidden_dim=128,
        decoder_activation='ReLU',
    )
    autoencoder.load_state_dict(torch.load('../data/pbmc_AE/model_seed=0_step=199999.pt'))
    return autoencoder

In [3]:
adata = sc.read_10x_mtx(
    '../data/pbmc68k/data/pbmc68k/filtered_matrices_mex/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)
adata.var_names_make_unique()
sc.pp.filter_cells(adata, min_genes=10)
sc.pp.filter_genes(adata, min_cells=3)
gene_names = adata.var_names
print(adata)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
################
celltype = pd.read_csv('../data/pbmc68k/data/pbmc68k/filtered_matrices_mex/68k_pbmc_barcodes_annotation.tsv', sep='\t')['celltype'].values
adata.obs['celltype'] = celltype
#########
cell_data = adata.X.toarray()[::5]
#generation
npzfile=np.load('../samples/pbmc/unconditional/pbmc.npz',allow_pickle=True)

cell_gen_all = npzfile['samples'][:10000]

AnnData object with n_obs × n_vars = 68579 × 17789
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'


In [4]:
autoencoder = load_VAE()
cell_gen_all = autoencoder(torch.tensor(cell_gen_all).cuda(),return_decoded=True).detach().cpu().numpy()
ori = ad.AnnData(cell_gen_all, dtype=np.float32)
cell_gen = ori.X

adata = np.concatenate((cell_data, cell_gen),axis=0)
adata = ad.AnnData(adata, dtype=np.float32)
adata.obs_names = [f"true_Cell" for i in range(cell_data.shape[0])]+[f"gen_Cell" for i in range(cell_gen.shape[0])]

sc.tl.pca(adata, n_comps=2, svd_solver='arpack')
real = adata[adata.obs_names=='true_Cell'].obsm['X_pca']
sim = adata[adata.obs_names=='gen_Cell'].obsm['X_pca']

data = np.concatenate((real,sim),axis=0)
label = np.concatenate((np.ones((real.shape[0])),np.zeros((sim.shape[0]))))

/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/_core/anndata.py:401: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/yjh/anaconda3/envs/scld/lib/python3.10/site-packages/anndata/_core/anndata.py:401: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import *
X_train,X_val,y_train,y_val = train_test_split(data, label,
                                               test_size = 0.25,random_state = 1)
print(X_train.shape)
rfc1 = RandomForestClassifier(n_estimators = 1000,
                              max_depth= 5,
                              oob_score=True,
                              class_weight = "balanced",
                              random_state=1)
rfc1.fit(X_train,y_train)
## accuracy
rfc1_lab = rfc1.predict(X_train)
rfc1_pre = rfc1.predict(X_val)

print("auc in validation set:",roc_auc_score(y_val,rfc1_pre))
mmd_value = compute_mmd(real, sim)
print("MMD value:", mmd_value)

(17787, 2)
auc in validation set: 0.5597464567573439
MMD value: 0.07157810916176348
